In [1]:
from bs4 import BeautifulSoup
import requests
import random

def get_ip_list(url, headers):
 web_data = requests.get(url, headers=headers)
 soup = BeautifulSoup(web_data.text, 'lxml')
 ips = soup.find_all('tr')
 ip_list = []
 for i in range(1, len(ips)):
  ip_info = ips[i]
  tds = ip_info.find_all('td')
  ip_list.append(tds[1].text + ':' + tds[2].text)
 return ip_list

def get_random_ip(ip_list):
 proxy_list = []
 for ip in ip_list:
  proxy_list.append('http://' + ip)
 proxy_ip = random.choice(proxy_list)
 proxies = {'http': proxy_ip}
 return proxies

if __name__ == '__main__':
 url = 'http://www.xicidaili.com/nn/'
 headers = {
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.60 Safari/537.17'
 }
 ip_list = get_ip_list(url, headers=headers)
 proxies = get_random_ip(ip_list)
 print(proxies)

{'http': 'http://117.69.168.205:4216'}


# 准备工作

In [2]:
'''
阿里研究院
阿里健康
阿里巴巴商学院
阿里数据

腾讯金融科技
腾讯研究院
腾讯媒体研究院
腾讯云启研究院
酷鹅用户研究院
'''
公众号 = "鳗鲸的海"

In [3]:
fn = { "output" : { "公众号_htm_snippets": "data_raw_src/公众号_htm_snippets_{公众号}.tsv",
                    "公众号_df": "data_raw_src/公众号_df_{公众号}.tsv",
                    "公众号_xlsx": "data_sets/公众号_url_{公众号}.xlsx" } \
      }

# 采集公众号（requests）

In [7]:
# 目标url

import time
import requests
import pandas as pd
import csv


url = "https://mp.weixin.qq.com/cgi-bin/appmsg"

# 使用Cookie，跳过登陆操作
headers = {
  "Cookie": "pgv_pvi=7968112640; RK=dAxoOkmNPM; ptcz=5e911a8738788675abb010f85db7a6c13b0570811db08ebefce3d01adfff5127; tvfe_boss_uuid=35a8107e1f02cbf7; pgv_pvid=160875890; o_cookie=576726974; pgv_si=s7389809664; uin=o0576726974; skey=@biD9HkZPV; ptisp=ctc; ua_id=o954FR4YG9aIylgbAAAAAAG1mFSkee-v8r6HcCv6PUM=; cert=xdo33QVRQxDDEATQOxVPYZVSl8hcgbBw; noticeLoginFlag=1; xid=; openid2ticket_omd36wPk8CgeWUiAccw6ZoK6cfKM=V9b+6gaDppAhuzkBsqQ2cqmVOoVbPExAl/fu+rqQ2xk=; mm_lang=zh_CN; rewardsn=; wxtokenkey=777; uuid=3f563116b647ce9a578c8d004d3508a8; ticket=a4fe0acfed322dd3eff3a301d94afded6dac3be0; ticket_id=gh_b8ce1853cd9b; rand_info=CAESIIpFhkwqorHCev9Vkx8d1xoS/h28VHdI65d1I+hS8rYh; slave_bizuin=3252904960; data_bizuin=3509374721; bizuin=3252904960; data_ticket=6z/ln+jO+jKPoA5MKzm3D6Cbfkg41cD9D6v6HTaHq8//2C9Hk73W6LtQlNMXteTT; slave_sid=eWpTNFJYU0hHQUFtcUt1TnBGSEZDYU55bHJXX2hhTTlZa3hvcno4MFRNaGd2QUlIT2Vxb0xHUmRWc1RQV2txQkI3OXFlUXhuOWY1aGVYbWdvTGFpNzNhTTdNaGhaRWNWNmwzTmFMRWlEa25Pa0VkZGpwVGkxNTRUdmRTbEtpbGF2UFUwbEZJR0lHdnlBcDFs; slave_user=gh_b8ce1853cd9b; openid2ticket_oAc8s0T6rPCr8qnMO7SoANf_3jVI=LJrk/Pum+BnDCZRPacwc+RgFKJv/+2uJEUtvOa1gdTU=",
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}

data = {
    "token": "1119521086",
    "lang": "zh_CN",
    "f": "json",
    "ajax": "1",
    "action": "list_ex",
    "begin": "0",
    "count": "5",
    "query": "",
    "fakeid": "MzI0Nzc3MTQyMw==",
    "type": "9",
}



content_list=[]

for i in range(5):
    data["begin"] = i*5
    print(data["begin"])
    time.sleep(3)
    # 使用get方法进行提交
    content_json = requests.get(url, headers=headers, params=data).json()
#     print(content_json)
    # 返回了一个json，里面是每一页的数据
    for item in content_json["app_msg_list"]:
    # 提取每页文章的标题及对应的url
        items = []
        items.append(item["title"])
        items.append(item["link"])
        items.append(item["create_time"])
        content_list.append(items)


name=['title','link','create_time']
test=pd.DataFrame(columns=name,data=content_list)
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号="蚂蚁金服科技_requests")) as writer:
    test.to_excel(writer)

# test.to_csv("../微信公众号爬虫_zhichao/南方周末.csv",mode='a',encoding='utf-8')
# print("保存成功")

0
5
10
15
20


NameError: name 'ExcelWriter' is not defined

# 采集公众号（selenium）

In [4]:
import pandas as pd
import numpy as np
from lxml.html import fromstring
import time
from random import random

# when selenium main_content is used
# Parses an HTML document from a string constant.  Returns the root nood
# root = fromstring(df.loc[1,"html_snippets"]) 

## 使用Selenium
* 要更改 opts.binary_location 至自己本地的Chrome浏览器，建议portable
* Chrome浏览器 和 chromedriver.exe要同版本号到小数后一位
* 要确保可以 开启浏览器机器人
* 要确保浏览器机器人 可以打开网页 driver.get("https://mp.weixin.qq.com")

In [5]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#caps=dict()
#caps["pageLoadStrategy"] = "none"   # Do not wait for full page load

opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')#解决DevToolsActivePort文件不存在的报错
opts.add_argument('window-size=1920x3000') #指定浏览器分辨率
opts.add_argument('--disable-gpu') #谷歌文档提到需要加上一这个属性来规避bug
opts.add_argument('--hide-scrollbars') #隐藏滚动条, 应对些特殊页面
#opts.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
#opts.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

opts.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" #"H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  

# "H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  
driver = webdriver.Chrome( chrome_options = opts) #desired_capabilities=caps,

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [6]:
driver.get("https://mp.weixin.qq.com")

## 填表登入

selenium 的定位方法
* find_element_by_id &ensp;&ensp;&ensp;  根据标签id定位
* find_element_by_name   &ensp;&ensp;&ensp; 根据标签的name定位
* find_element_by_xpath  &ensp;&ensp;&ensp; 根据xpath定位
* find_element_by_link_text  &ensp;&ensp;&ensp; 通过文字链接来定位元素
* find_element_by_partial_link_text  &ensp;&ensp;&ensp;  通过文字链接来定位元素
* find_element_by_tag_name  &ensp;&ensp;&ensp;  根据标签的名字定位
* find_element_by_class_name  &ensp;&ensp;&ensp; 通过class name 定位
* find_element_by_css_selector  &ensp;&ensp;&ensp;  根据元素属性来定位

In [7]:
payload =  {"account": "penwey@126.com", "password": "penwey336699"}
# payload =  {"account": "NFUHacks@163.com", "password": "NFU706947580"}
driver.find_element_by_xpath('//div[@class="login__type__container login__type__container__scan"]/a').click()

WebDriver 常用方法：
* clear()清楚文本
* send_keys(values)模拟按键输入
* click()模拟点击
* submit模拟提交

In [8]:
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').send_keys(payload['account'])
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').send_keys(payload['password'])

In [9]:
driver.find_element_by_xpath('//div[@class="login_btn_panel"]/a').click()

## 点选单

其他常用方法
* size：返回元素的尺寸
* text：获取元素的文本
* get_attribute：获取属性值  &ensp;&ensp;&ensp; get_attribute('innerHTML')获取元素内的全部HTML
* is_displayed()：设置该元素用户是否可见

In [10]:
element = driver.find_element_by_xpath('//a[@id="m_open"]')
element.click()
main_content = element.get_attribute('innerHTML')
main_content

'展开'

In [11]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [12]:
element = driver.find_element_by_xpath('//li[@title[contains(.,"素材管理")]]/a') 
# main_content = element.get_attribute('innerHTML')
# main_content
url2= element.get_attribute("href")
url2

'https://mp.weixin.qq.com/cgi-bin/appmsg?begin=0&count=10&t=media/appmsg_list&type=10&action=list&token=1053839229&lang=zh_CN'

In [13]:
driver.get(url2)

## 新建图文消息

In [14]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"新建图文消息")]]') 
main_content = element.get_attribute('innerHTML')
main_content
element.click()

In [15]:
print (driver.window_handles)

['CDwindow-D01F928EF4BE46A1F534707AEC37F718', 'CDwindow-56D5C9F4A8F836111A5E35C374B1BDE0']


In [16]:
# 新建图文消息开了另一分视窗，所以要切换 switch_to 
driver.switch_to.window(driver.window_handles[-1])

## 超链接

In [17]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"超链接")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

                超链接              


In [18]:
# 点 选择其他公众号
element = driver.find_element_by_xpath('//*[text()[contains(.,"选择其他公众号")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

选择其他公众号


In [19]:
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').clear()
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').send_keys(公众号)

In [20]:
# 点放大镜搜
element = driver.find_element_by_xpath('//button[@class="weui-desktop-icon-btn weui-desktop-search__btn"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-icon weui-desktop-icon__inputSearch weui-desktop-icon__small"><!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <svg width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><path d="M11.33 10.007l4.273 4.273a.502.502 0 0 1 .005.709l-.585.584a.499.499 0 0 1-.709-.004L10.046 11.3a6.278 6.278 0 1 1 1.284-1.294zm.012-3.729a5.063 5.063 0 1 0-10.127 0 5.063 5.063 0 0 0 10.127 0z"></path></svg> <!----> <!----> <!----> <!----></div>


In [21]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
公众号SERP = main_content


<li class="inner_link_account_item"><div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/brfCPu4x1r74UH78vRicwJW7Jkx27Rlq85m2FkO8AxiaxXEIM9ox9iccvbvvyRwHu0Oq0ibO2q8HckhY6lFkykFu6A/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">鳗鲸的海</strong> <i class="inner_link_account_wechat">微信号：manjing2017</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div></li>


In [22]:
# 解析
root = fromstring(公众号SERP) 

In [23]:
主 = root.xpath('//li[@class="inner_link_account_item"]')

account_list = []
for e in 主:
    account_nickname = e.xpath('./div/strong[@class="inner_link_account_nickname"]')[0].text
    account_wechat = e.xpath('./div/i[@class="inner_link_account_wechat"]')[0].text
    account_img = e.xpath('./div/img/@src')[0]
    account = {"nickname": account_nickname, "wechat": account_wechat, "img": account_img,}
    account_list.append(account)

df_account = pd.DataFrame(account_list)

In [24]:
df_account

,nickname,wechat,img
0,鳗鲸的海,微信号：manjing2017,http://mmbiz.qpic.cn/mmbiz_png/brfCPu4x1r74UH7...


In [25]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]/li')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/brfCPu4x1r74UH78vRicwJW7Jkx27Rlq85m2FkO8AxiaxXEIM9ox9iccvbvvyRwHu0Oq0ibO2q8HckhY6lFkykFu6A/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">鳗鲸的海</strong> <i class="inner_link_account_wechat">微信号：manjing2017</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div>


In [26]:
# 跳转testing
'''
跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
跳转_input.clear()
跳转_input.send_keys(2)
跳转_a.click()
'''

'\n跳转_input = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/input\')\n跳转_a = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/a\')\n跳转_input.clear()\n跳转_input.send_keys(2)\n跳转_a.click()\n'

In [27]:
# 跳转上限
l_e = driver.find_elements_by_xpath('//label[@class="weui-desktop-pagination__num"]')
l_e_int  = [int(x.text) for x in l_e] 
print (l_e_int)
print (l_e_int[0]==l_e_int[-1])

[1, 85]
False


In [28]:
pages = list(range(l_e_int[0],l_e_int[-1]+1 ))
#print(pages[0:2])
pages = list(range(1,l_e_int[-1]+1 ))
print(pages)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]


## 循环/遍历

In [29]:
# global varialbes 
html_raw = dict()
main_content =""
element = None

In [30]:
def process_pages (pages):
    for p in pages:
        print (p,end='\t')

        跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
        跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
        跳转_input.clear()
        跳转_input.send_keys(p)
        跳转_a.click()

        time.sleep(45+120*random())

        element = driver.find_element_by_xpath('//div[@class="inner_link_article_list"]')
        main_content = element.get_attribute('innerHTML')
        #print(main_content)
        html_raw[p] = main_content

In [31]:
process_pages(pages)

1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	

In [33]:
df = pd.DataFrame([html_raw]).T
df.columns = ["html_snippets"]
df

,html_snippets
1,"<div><label class=""inner_link_article_item""><s..."
2,"<div><label class=""inner_link_article_item""><s..."
3,"<div><label class=""inner_link_article_item""><s..."
4,"<div><label class=""inner_link_article_item""><s..."
5,"<div><label class=""inner_link_article_item""><s..."
...,...
81,"<div><label class=""inner_link_article_item""><s..."
82,"<div><label class=""inner_link_article_item""><s..."
83,"<div><label class=""inner_link_article_item""><s..."
84,"<div><label class=""inner_link_article_item""><s..."


In [34]:
%store html_raw
import pickle 
filehandler = open("html_raw", 'wb') 
pickle.dump(html_raw, filehandler)

Stored 'html_raw' (dict)


In [35]:
df_out = df[~df.duplicated()]
print (len(df_out))
df[df.duplicated()]

84


,html_snippets
12,"<div><label class=""inner_link_article_item""><s..."


In [36]:
try_again = list(df[df.duplicated()].index)
print(try_again)
try_again = try_again + list (set(pages).difference(set(df.index.values)))
try_again

[12]


[12]

## 暂存档

In [37]:
filename = fn ["output"] ["公众号_htm_snippets"] 
df_out.to_csv(filename.format(公众号=公众号), sep="\t", encoding="utf8")

In [38]:
def parse_html_snippets(_snippet_):
    root = fromstring(_snippet_) 
    title = [x.text for x in root.xpath('//div[@class="inner_link_article_title"]')]
    create_time = [x.text for x in root.xpath('//div[@class="inner_link_article_date"]')]
    link = [x for x in root.xpath('//a/@href')]
    _df_ = pd.DataFrame({"title":title, "create_time": create_time, "link":link})
    return(_df_)
    
l_df = []
for p in pages:
    _df_ = parse_html_snippets(df.loc[p,"html_snippets"])
    print (len(_df_), end=",")
    l_df.append(_df_)

5,5,5,5,6,5,5,5,5,5,5,5,5,5,5,5,5,6,5,5,5,5,5,5,5,5,5,5,5,5,5,6,5,5,9,7,6,9,6,8,7,7,9,8,7,5,5,5,5,5,5,5,5,5,6,6,5,5,5,5,5,6,5,6,7,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,

In [39]:
df_url_out = pd.concat(l_df).reset_index(drop=True)
df_url_out.loc[0:10]

,title,create_time,link
0,那些不让前度死去的人啊。,2020-05-13,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
1,人要迅速爱上另一个人，也不难吧。,2020-05-05,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
2,一个人在世上，最好要有两个家。,2020-05-03,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
3,今日跟朋友聊到 \n很长一段时间自己无论做些什么 \n甚至只是说点什么 \n都会被过度解读 ...,2020-04-14,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
4,原来我没有能想念的人了。,2020-04-12,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
5,我们把「相亲实验试」搬到了你家门口。,2019-07-26,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
6,那男人对我说：,2019-07-25,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
7,我们在炎热与抑郁的夏天\n无法停止写诗\n\n/ 广州雨夜 我与贱贱 \n,2019-07-23,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
8,昨天在隆哥老司机的指引下 我俩穿着球鞋去蹦迪 感受了一回什么叫醉生梦死\n\n刚醒来 广州又...,2019-07-19,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
9,我很想念那个迷死人的你。,2019-07-17,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...


In [40]:
df_url_out.tail(5)

,title,create_time,link
457,志明与春娇，这次你们真的分手吧。,2017-03-10,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
458,像你这么笨，根本不懂如何安慰人。,2017-03-09,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
459,《遗物收藏馆》| 你还以为你忘不掉了？,2017-03-08,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
460,他爱的是朋友圈，不是你。,2017-03-07,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
461,你是不是也想过，余生不再谈恋爱？,2017-03-01,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...


In [41]:
# tagging 标记
tagging_list = ["","马化腾", "腾讯", "微众银行","腾讯复星","腾讯风控","腾讯支付","WeChat","We Remit",\
                "公益日","红包",\
                "腾讯财付通","鹅厂","QQ钱包","QQ红包","QQ",\
                "只有一","大咖","听说","图片","照片","小编",\
                "洗钱", "黑产","被骗","腾讯安全课","诈骗", "炒股","神秘兼职","神秘组织",\
                "财付通", "品牌",\
                "收款",\
                "报告",\
                "银行卡","理财","选股","发票","基金",\
                "区块链","金融云","O2O","农产品","家乡",\
                "数据", "数据赋能", "智能", "数字孪生", "智慧大脑",\
                "出行","乘车","公交","乘车码", "智慧地铁",\
                "高峰论坛", "智库",\
                "央行","新规", \
                "微信", "微信支付", "跨境支付", "移动支付","非银行支付","电子支付",\
                "互联网金融", "金融科技","互联网＋","互联网+金融","普惠金融","虚拟银行",\
                "开放","生态","复杂","互联网思维","全球合作伙伴",\
                "联合国", "城市", "粤港澳大湾区", "平台", "可持续发展", "未来", "绿色",\
                "医护","防护服","小时","武汉","危机","新冠肺炎", "疫", "疫情", "复工","停课","宅经济",\
                "基建","新基建"] #overwritable

v_v_list = []

for tag in tagging_list:
    index_list = df_url_out [ df_url_out.title.str.contains(tag) ].index.tolist()
    v_v_pairs = pd.DataFrame({tag:index_list}).melt().set_index("value")
    v_v_list.append(v_v_pairs)

df_cat = v_v_list[0]
for d in v_v_list:
    df_cat.update(d)
    
# 尚未标记内容
df_url_out.loc [ df_cat.query('variable==""').index ]

,title,create_time,link
value,,,
0,那些不让前度死去的人啊。,2020-05-13,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
1,人要迅速爱上另一个人，也不难吧。,2020-05-05,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
2,一个人在世上，最好要有两个家。,2020-05-03,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
3,今日跟朋友聊到 \n很长一段时间自己无论做些什么 \n甚至只是说点什么 \n都会被过度解读 ...,2020-04-14,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
4,原来我没有能想念的人了。,2020-04-12,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
...,...,...,...
457,志明与春娇，这次你们真的分手吧。,2017-03-10,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
458,像你这么笨，根本不懂如何安慰人。,2017-03-09,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
459,《遗物收藏馆》| 你还以为你忘不掉了？,2017-03-08,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...


In [42]:
df_url_out[df_url_out.duplicated()]

,title,create_time,link
56,我们把「相亲实验试」搬到了你家门口。,2019-07-26,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
57,那男人对我说：,2019-07-25,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
58,我们在炎热与抑郁的夏天\n无法停止写诗\n\n/ 广州雨夜 我与贱贱 \n,2019-07-23,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
59,昨天在隆哥老司机的指引下 我俩穿着球鞋去蹦迪 感受了一回什么叫醉生梦死\n\n刚醒来 广州又...,2019-07-19,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
60,我很想念那个迷死人的你。,2019-07-17,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...


In [43]:
df_url_out[~df_url_out.duplicated()]

,title,create_time,link
0,那些不让前度死去的人啊。,2020-05-13,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
1,人要迅速爱上另一个人，也不难吧。,2020-05-05,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
2,一个人在世上，最好要有两个家。,2020-05-03,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
3,今日跟朋友聊到 \n很长一段时间自己无论做些什么 \n甚至只是说点什么 \n都会被过度解读 ...,2020-04-14,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
4,原来我没有能想念的人了。,2020-04-12,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
...,...,...,...
457,志明与春娇，这次你们真的分手吧。,2017-03-10,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
458,像你这么笨，根本不懂如何安慰人。,2017-03-09,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
459,《遗物收藏馆》| 你还以为你忘不掉了？,2017-03-08,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...
460,他爱的是朋友圈，不是你。,2017-03-07,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...


In [44]:
df_o = df_url_out.join(df_cat).replace("", np.nan).fillna("无法分类")
df_o

,title,create_time,link,variable
0,那些不让前度死去的人啊。,2020-05-13,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
1,人要迅速爱上另一个人，也不难吧。,2020-05-05,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
2,一个人在世上，最好要有两个家。,2020-05-03,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
3,今日跟朋友聊到 \n很长一段时间自己无论做些什么 \n甚至只是说点什么 \n都会被过度解读 ...,2020-04-14,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
4,原来我没有能想念的人了。,2020-04-12,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
...,...,...,...,...
457,志明与春娇，这次你们真的分手吧。,2017-03-10,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
458,像你这么笨，根本不懂如何安慰人。,2017-03-09,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
459,《遗物收藏馆》| 你还以为你忘不掉了？,2017-03-08,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类
460,他爱的是朋友圈，不是你。,2017-03-07,http://mp.weixin.qq.com/s?__biz=MzAxNDMxMjgyOA...,无法分类


In [45]:
df_o[df_o.title.str.contains("腾讯")]

,title,create_time,link,variable


In [46]:
df_stats = df_o.groupby(by="variable").agg({"title":"count"}).sort_values(by="title", ascending=False)
df_stats

,title
variable,
无法分类,416
图片,32
微信,5
听说,3
城市,3
照片,2
疫,1


## 输出

In [47]:
df_account.columns.name = "rel_accounts"
df_o.columns.name = "url_cat"
df_stats.columns.name = "stats"


In [48]:
_df_.columns.name

In [49]:
# Get the xlsxwriter workbook and worksheet objects.  
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号=公众号)) as writer:
    workbook  = writer.book

    for _df_ in [df_account, df_o, df_stats]:
        _df_.to_excel(writer, sheet_name = _df_.columns.name)
